# Uso de UDFs: Funciones Personalizadas en Spark

Las UDF permiten aplicar funciones de Python a tus DataFrames. Úsalas solo cuando las funciones nativas no sean suficientes.
Aquí, analizamos los títulos de películas del dataset MovieLens.


In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

# Cargar datos de películas
movies = spark.read.csv("/dbfs/FileStore/movielens/movies.csv", header=True, inferSchema=True)

# Definir UDF: contar número de palabras en el título de cada película
def contar_palabras(titulo):
    return len(titulo.split(" ")) if titulo else 0

contar_palabras_udf = udf(contar_palabras, IntegerType())

# Aplicar UDF
movies = movies.withColumn("num_palabras_titulo", contar_palabras_udf(col("title")))
movies.select("title", "num_palabras_titulo").show(10, truncate=False)

# Comparar con una función Spark nativa (split)
from pyspark.sql.functions import size, split
movies = movies.withColumn("num_palabras_nativo", size(split(col("title"), " ")))
movies.select("title", "num_palabras_titulo", "num_palabras_nativo").show(10, truncate=False)

# Nota: siempre que sea posible, prefiere las funciones Spark nativas por rendimiento.
